In [ ]:
# === Setup ===
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from pathlib import Path
import sys
import time

sys.path.insert(0, str(Path.cwd().parent.parent.parent))

from modules._import_helper import safe_import_from

set_seed = safe_import_from('00_repo_standards.src.mlphys_core', 'set_seed')
PINN, PINNConfig, compute_gradient = safe_import_from(
    '07_physics_informed_ml.src.pinn_base',
    'PINN', 'PINNConfig', 'compute_gradient'
)

plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 11
plt.rcParams['axes.grid'] = True
plt.rcParams['grid.alpha'] = 0.3

reports_dir = Path.cwd().parent / 'reports'
reports_dir.mkdir(exist_ok=True)

SEED = 42
set_seed(SEED)
torch.manual_seed(SEED)

print("✓ Setup complete")

---
## 1. The Inverse Problem

### Damped Harmonic Oscillator

$$\ddot{x} + 2\gamma\dot{x} + \omega^2 x = 0$$

**Forward problem**: Given $(\gamma, \omega)$, find $x(t)$

**Inverse problem**: Given noisy observations $\{(t_i, x_i)\}$, find $\gamma$

### Analytical Solution (Underdamped: $\gamma < \omega$)

$$x(t) = e^{-\gamma t} \left[ A \cos(\omega_d t) + B \sin(\omega_d t) \right]$$

where $\omega_d = \sqrt{\omega^2 - \gamma^2}$

### Why Is This Hard?

1. **Non-convex**: Multiple local minima in parameter space
2. **Ill-conditioned**: Small data changes → large parameter changes
3. **Noise sensitivity**: Noise in $x$ propagates to $\gamma$ estimate

In [ ]:
# === Generate synthetic data ===
def damped_oscillator_analytical(omega, gamma, x0, v0, t):
    """Analytical solution for underdamped oscillator."""
    omega_d = np.sqrt(omega**2 - gamma**2)
    A = x0
    B = (v0 + gamma * x0) / omega_d
    x = np.exp(-gamma * t) * (A * np.cos(omega_d * t) + B * np.sin(omega_d * t))
    return x

# True parameters (we want to recover gamma)
omega_true = 2.0      # Known angular frequency
gamma_true = 0.3      # UNKNOWN damping coefficient
x0, v0 = 1.0, 0.0     # Initial conditions

# Generate observations
np.random.seed(SEED)
n_obs = 30
t_obs = np.sort(np.random.uniform(0, 10, n_obs))
x_clean = damped_oscillator_analytical(omega_true, gamma_true, x0, v0, t_obs)

# Add noise
noise_level = 0.05
x_obs = x_clean + noise_level * np.random.randn(n_obs)

# Dense evaluation for plotting
t_dense = np.linspace(0, 10, 200)
x_true = damped_oscillator_analytical(omega_true, gamma_true, x0, v0, t_dense)

# Visualize
plt.figure(figsize=(10, 5))
plt.plot(t_dense, x_true, 'k-', lw=2, label='True solution')
plt.scatter(t_obs, x_obs, c='r', s=50, zorder=5, label=f'Observations (n={n_obs})')
plt.xlabel('Time t')
plt.ylabel('Position x')
plt.title(f'Inverse Problem Setup\nTrue γ = {gamma_true} (unknown), ω = {omega_true} (known)')
plt.legend()
plt.savefig(reports_dir / '05_inverse_problem_setup.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"Observations: {n_obs} points")
print(f"Noise level: {noise_level} (σ ≈ {noise_level * np.std(x_clean):.4f})")
print(f"\nGoal: Recover γ_true = {gamma_true}")

---
## 2. Inverse PINN Formulation

### Architecture

**Trainable variables**:
1. NN weights $\theta$ (predict $\hat{x}(t)$)
2. Unknown parameter $\gamma$ (scalar)

### Loss Function

$$\mathcal{L} = \mathcal{L}_{\text{data}} + \lambda_1 \mathcal{L}_{\text{physics}} + \lambda_2 \mathcal{L}_{\text{IC}}$$

where:

$$\mathcal{L}_{\text{data}} = \frac{1}{N_{\text{obs}}} \sum_i (\hat{x}(t_i) - x_i^{\text{obs}})^2$$

$$\mathcal{L}_{\text{physics}} = \frac{1}{N_c} \sum_j \left| \ddot{\hat{x}}(t_j) + 2\gamma \dot{\hat{x}}(t_j) + \omega^2 \hat{x}(t_j) \right|^2$$

**Key**: $\gamma$ appears in $\mathcal{L}_{\text{physics}}$ and is optimized jointly!

In [ ]:
# === Inverse PINN Implementation ===

class InversePINN:
    """
    PINN for inverse problem: identify damping coefficient γ.
    
    ODE: x'' + 2*gamma*x' + omega^2*x = 0
    Known: omega, initial conditions, sparse observations
    Unknown: gamma (learned as trainable parameter)
    """
    
    def __init__(
        self,
        omega: float,
        x0: float,
        v0: float,
        t_max: float,
        gamma_init: float = 0.1,  # Initial guess for gamma
        n_collocation: int = 200,
        hidden_dims: list = [64, 64, 64],
        epochs: int = 5000,
        lr: float = 1e-3,
        lr_gamma: float = 1e-2,  # Separate LR for gamma
        lambda_physics: float = 1.0,
        lambda_ic: float = 10.0,
    ):
        self.omega = omega
        self.x0 = x0
        self.v0 = v0
        self.t_max = t_max
        self.epochs = epochs
        self.lambda_physics = lambda_physics
        self.lambda_ic = lambda_ic
        
        # Neural network for x(t)
        pinn_config = PINNConfig(
            input_dim=1, output_dim=1,
            hidden_dims=hidden_dims,
            activation='tanh'
        )
        self.model = PINN(pinn_config)
        
        # TRAINABLE parameter gamma
        self.gamma = nn.Parameter(torch.tensor([gamma_init], dtype=torch.float32))
        
        # Optimizers
        self.optimizer_nn = torch.optim.Adam(self.model.parameters(), lr=lr)
        self.optimizer_gamma = torch.optim.Adam([self.gamma], lr=lr_gamma)
        
        # Collocation points
        self.t_col = torch.linspace(0, t_max, n_collocation).view(-1, 1)
        self.t_col.requires_grad = True
        
        # IC point
        self.t_ic = torch.tensor([[0.0]], requires_grad=True)
    
    def physics_residual(self, t):
        """Compute ODE residual with current gamma estimate."""
        x = self.model(t)
        x_t = compute_gradient(x, t, order=1)
        x_tt = compute_gradient(x, t, order=2)
        
        # r = x'' + 2*gamma*x' + omega^2*x
        residual = x_tt + 2 * self.gamma * x_t + self.omega**2 * x
        return residual
    
    def train(self, t_obs, x_obs, verbose=500):
        """Train with observations."""
        t_obs_tensor = torch.tensor(t_obs, dtype=torch.float32).view(-1, 1)
        x_obs_tensor = torch.tensor(x_obs, dtype=torch.float32).view(-1, 1)
        
        history = {
            'loss': [], 'loss_data': [], 'loss_physics': [], 'loss_ic': [],
            'gamma': []
        }
        
        self.model.train()
        
        for epoch in range(self.epochs):
            self.optimizer_nn.zero_grad()
            self.optimizer_gamma.zero_grad()
            
            # Data loss
            x_pred_obs = self.model(t_obs_tensor)
            loss_data = torch.mean((x_pred_obs - x_obs_tensor)**2)
            
            # Physics loss
            residual = self.physics_residual(self.t_col)
            loss_physics = torch.mean(residual**2)
            
            # IC loss
            x_ic = self.model(self.t_ic)
            v_ic = compute_gradient(x_ic, self.t_ic, order=1)
            loss_ic = (x_ic - self.x0)**2 + (v_ic - self.v0)**2
            loss_ic = loss_ic.squeeze()
            
            # Total loss
            loss = loss_data + self.lambda_physics * loss_physics + self.lambda_ic * loss_ic
            
            loss.backward()
            self.optimizer_nn.step()
            self.optimizer_gamma.step()
            
            # Enforce gamma > 0 (physical constraint)
            with torch.no_grad():
                self.gamma.clamp_(min=0.001)
            
            # Record
            history['loss'].append(loss.item())
            history['loss_data'].append(loss_data.item())
            history['loss_physics'].append(loss_physics.item())
            history['loss_ic'].append(loss_ic.item())
            history['gamma'].append(self.gamma.item())
            
            if verbose > 0 and (epoch + 1) % verbose == 0:
                print(f"Epoch {epoch+1:5d} | Loss: {loss.item():.6f} | "
                      f"γ_est: {self.gamma.item():.4f}")
        
        return history
    
    def predict(self, t):
        """Predict x(t)."""
        self.model.eval()
        with torch.no_grad():
            t_tensor = torch.tensor(t, dtype=torch.float32).view(-1, 1)
            return self.model(t_tensor).numpy().flatten()
    
    def get_gamma(self):
        """Return estimated gamma."""
        return self.gamma.item()

In [ ]:
# === Train inverse PINN ===
set_seed(SEED)
torch.manual_seed(SEED)

inv_pinn = InversePINN(
    omega=omega_true,
    x0=x0, v0=v0,
    t_max=10.0,
    gamma_init=0.1,  # Initial guess (intentionally wrong)
    n_collocation=300,
    hidden_dims=[64, 64, 64],
    epochs=8000,
    lr=1e-3,
    lr_gamma=5e-3,
    lambda_physics=1.0,
    lambda_ic=10.0,
)

print(f"Initial γ guess: {inv_pinn.get_gamma():.4f}")
print(f"True γ: {gamma_true}")
print("\nTraining...")

history = inv_pinn.train(t_obs, x_obs, verbose=1000)

gamma_est = inv_pinn.get_gamma()
gamma_error = abs(gamma_est - gamma_true)
rel_error = gamma_error / gamma_true * 100

print("\n" + "="*50)
print("PARAMETER IDENTIFICATION RESULT")
print("="*50)
print(f"True γ:      {gamma_true:.4f}")
print(f"Estimated γ: {gamma_est:.4f}")
print(f"Error:       {gamma_error:.4f} ({rel_error:.2f}%)")
print("="*50)

In [ ]:
# === Visualization ===
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Training convergence
ax = axes[0, 0]
ax.semilogy(history['loss'], 'k-', lw=2, label='Total')
ax.semilogy(history['loss_data'], 'b--', alpha=0.7, label='Data')
ax.semilogy(history['loss_physics'], 'r:', alpha=0.7, label='Physics')
ax.set_xlabel('Epoch')
ax.set_ylabel('Loss')
ax.set_title('Training Convergence')
ax.legend()

# Gamma evolution
ax = axes[0, 1]
ax.plot(history['gamma'], 'b-', lw=2)
ax.axhline(gamma_true, color='r', linestyle='--', lw=2, label=f'True γ = {gamma_true}')
ax.set_xlabel('Epoch')
ax.set_ylabel('γ estimate')
ax.set_title('Parameter Learning Trajectory')
ax.legend()

# Solution comparison
ax = axes[1, 0]
x_pred = inv_pinn.predict(t_dense)
ax.plot(t_dense, x_true, 'k-', lw=2, label='True (γ_true)')
ax.plot(t_dense, x_pred, 'b--', lw=2, label=f'PINN (γ_est={gamma_est:.4f})')
ax.scatter(t_obs, x_obs, c='r', s=50, zorder=5, label='Observations')
ax.set_xlabel('Time t')
ax.set_ylabel('x(t)')
ax.set_title('Solution Comparison')
ax.legend()

# Solution with wrong gamma
ax = axes[1, 1]
x_wrong = damped_oscillator_analytical(omega_true, 0.1, x0, v0, t_dense)  # Wrong gamma
ax.plot(t_dense, x_true, 'k-', lw=2, label=f'True (γ={gamma_true})')
ax.plot(t_dense, x_pred, 'b--', lw=2, label=f'PINN (γ={gamma_est:.3f})')
ax.plot(t_dense, x_wrong, 'g:', lw=2, alpha=0.7, label='Initial guess (γ=0.1)')
ax.set_xlabel('Time t')
ax.set_ylabel('x(t)')
ax.set_title('Effect of Correct vs Wrong γ')
ax.legend()

plt.tight_layout()
plt.savefig(reports_dir / '05_inverse_pinn_result.png', dpi=150, bbox_inches='tight')
plt.show()

---
## 3. Experiment: Sensitivity to Noise

In [ ]:
# === Noise sensitivity ===
noise_levels = [0.01, 0.05, 0.1, 0.2]
results_noise = []

print("EXPERIMENT: Sensitivity to Noise")
print("="*60)

for noise in noise_levels:
    set_seed(SEED)
    torch.manual_seed(SEED)
    
    # Generate noisy data
    x_noisy = x_clean + noise * np.random.randn(n_obs)
    
    # Train
    inv_pinn_test = InversePINN(
        omega=omega_true, x0=x0, v0=v0, t_max=10.0,
        gamma_init=0.1, epochs=5000, lr_gamma=5e-3
    )
    _ = inv_pinn_test.train(t_obs, x_noisy, verbose=0)
    
    gamma_est = inv_pinn_test.get_gamma()
    error = abs(gamma_est - gamma_true)
    rel_error = error / gamma_true * 100
    
    results_noise.append({
        'noise': noise,
        'gamma_est': gamma_est,
        'error': error,
        'rel_error': rel_error,
    })
    
    print(f"Noise={noise:.2f} | γ_est={gamma_est:.4f} | Error={error:.4f} ({rel_error:.1f}%)")

# Plot
fig, ax = plt.subplots(figsize=(8, 5))
noises = [r['noise'] for r in results_noise]
errors = [r['error'] for r in results_noise]
ax.plot(noises, errors, 'ko-', lw=2, ms=10)
ax.set_xlabel('Noise Level')
ax.set_ylabel('|γ_est - γ_true|')
ax.set_title('Parameter Estimation Error vs Noise')
plt.tight_layout()
plt.savefig(reports_dir / '05_noise_sensitivity.png', dpi=150, bbox_inches='tight')
plt.show()

---
## 4. Experiment: Sensitivity to Data Sparsity

In [ ]:
# === Data sparsity ===
n_obs_values = [10, 20, 30, 50, 100]
results_sparsity = []

print("EXPERIMENT: Sensitivity to Data Sparsity")
print("="*60)

for n in n_obs_values:
    set_seed(SEED)
    torch.manual_seed(SEED)
    np.random.seed(SEED)
    
    # Generate data
    t_sparse = np.sort(np.random.uniform(0, 10, n))
    x_clean_sparse = damped_oscillator_analytical(omega_true, gamma_true, x0, v0, t_sparse)
    x_sparse = x_clean_sparse + 0.05 * np.random.randn(n)
    
    # Train
    inv_pinn_test = InversePINN(
        omega=omega_true, x0=x0, v0=v0, t_max=10.0,
        gamma_init=0.1, epochs=5000, lr_gamma=5e-3
    )
    _ = inv_pinn_test.train(t_sparse, x_sparse, verbose=0)
    
    gamma_est = inv_pinn_test.get_gamma()
    error = abs(gamma_est - gamma_true)
    
    results_sparsity.append({
        'n_obs': n,
        'gamma_est': gamma_est,
        'error': error,
    })
    
    print(f"N_obs={n:3d} | γ_est={gamma_est:.4f} | Error={error:.4f}")

# Plot
fig, ax = plt.subplots(figsize=(8, 5))
ns = [r['n_obs'] for r in results_sparsity]
errors = [r['error'] for r in results_sparsity]
ax.plot(ns, errors, 'ko-', lw=2, ms=10)
ax.set_xlabel('Number of Observations')
ax.set_ylabel('|γ_est - γ_true|')
ax.set_title('Parameter Estimation Error vs Data Amount')
plt.tight_layout()
plt.savefig(reports_dir / '05_sparsity_sensitivity.png', dpi=150, bbox_inches='tight')
plt.show()

---
## 5. Experiment: Sensitivity to Initial Guess

In [ ]:
# === Initial guess sensitivity ===
gamma_inits = [0.05, 0.1, 0.2, 0.5, 0.8]
results_init = []

print("EXPERIMENT: Sensitivity to Initial Guess")
print(f"True γ = {gamma_true}")
print("="*60)

for g_init in gamma_inits:
    set_seed(SEED)
    torch.manual_seed(SEED)
    
    inv_pinn_test = InversePINN(
        omega=omega_true, x0=x0, v0=v0, t_max=10.0,
        gamma_init=g_init,
        epochs=5000, lr_gamma=5e-3
    )
    history_test = inv_pinn_test.train(t_obs, x_obs, verbose=0)
    
    gamma_est = inv_pinn_test.get_gamma()
    error = abs(gamma_est - gamma_true)
    
    results_init.append({
        'gamma_init': g_init,
        'gamma_est': gamma_est,
        'error': error,
        'history': history_test['gamma'],
    })
    
    print(f"γ_init={g_init:.2f} | γ_est={gamma_est:.4f} | Error={error:.4f}")

# Plot learning trajectories
fig, ax = plt.subplots(figsize=(10, 5))
colors = plt.cm.viridis(np.linspace(0, 1, len(results_init)))
for i, r in enumerate(results_init):
    ax.plot(r['history'], color=colors[i], lw=2, label=f"γ_init={r['gamma_init']:.2f}")
ax.axhline(gamma_true, color='r', linestyle='--', lw=2, label=f'True γ={gamma_true}')
ax.set_xlabel('Epoch')
ax.set_ylabel('γ estimate')
ax.set_title('Parameter Learning from Different Initial Guesses')
ax.legend()
plt.tight_layout()
plt.savefig(reports_dir / '05_initial_guess_sensitivity.png', dpi=150, bbox_inches='tight')
plt.show()

---
## 6. Summary Results

In [ ]:
# === Save summary ===
summary = f"""
# Inverse Problem: Parameter Identification - Results Summary

**Date**: {time.strftime('%Y-%m-%d %H:%M')}
**Seed**: {SEED}

## Problem Setup

- ODE: x'' + 2γx' + ω²x = 0 (damped oscillator)
- Known: ω = {omega_true}
- Unknown: γ (true value = {gamma_true})
- Observations: {n_obs} noisy measurements

## Main Result

| Metric | Value |
|--------|-------|
| True γ | {gamma_true:.4f} |
| Estimated γ | {gamma_est:.4f} |
| Absolute Error | {gamma_error:.4f} |
| Relative Error | {rel_error:.2f}% |

## Noise Sensitivity

| Noise Level | γ_est | Error |
|-------------|-------|-------|
""" + "\n".join([f"| {r['noise']:.2f} | {r['gamma_est']:.4f} | {r['error']:.4f} |" for r in results_noise]) + """

## Data Sparsity Sensitivity

| N_obs | γ_est | Error |
|-------|-------|-------|
""" + "\n".join([f"| {r['n_obs']} | {r['gamma_est']:.4f} | {r['error']:.4f} |" for r in results_sparsity]) + """

## Initial Guess Sensitivity

| γ_init | γ_est | Error |
|--------|-------|-------|
""" + "\n".join([f"| {r['gamma_init']:.2f} | {r['gamma_est']:.4f} | {r['error']:.4f} |" for r in results_init]) + """

## Key Findings

1. **PINN successfully identifies damping coefficient** from sparse noisy data
2. **Noise increases error** but physics constraint provides regularization
3. **More data helps** but even 10 points give reasonable estimates
4. **Initial guess matters less** than for pure optimization (NN finds solution path)
5. **Physics constraint is key**: Without it, inverse problem is ill-posed

## When Parameter ID Works

✅ **Good conditions**:
- Parameter affects solution significantly (sensitivity)
- Physics model is correct
- Data spans enough dynamics

❌ **Challenging conditions**:
- Parameters are weakly identifiable
- Very high noise
- Model mismatch (wrong physics)

## Comparison to Classical Methods

| Method | Advantages | Disadvantages |
|--------|------------|---------------|
| Least squares | Fast, well-understood | Needs analytical gradients |
| PINN | Flexible, mesh-free | Slower, hyperparameter tuning |
| Bayesian | Uncertainty quantification | Computationally expensive |
"""

with open(reports_dir / '05_inverse_problem_summary.md', 'w') as f:
    f.write(summary)

print(summary)
print(f"\n✓ Summary saved to {reports_dir / '05_inverse_problem_summary.md'}")

---
## 7. Mini Exercises

**Exercise 1**: Try identifying ω instead of γ (assume γ is known). Is it easier or harder?

**Exercise 2**: Identify BOTH γ and ω simultaneously. Does it work?

**Exercise 3**: Add a simple uncertainty estimate using bootstrap (train multiple times with different subsets).

**Exercise 4**: What happens if the physics model is wrong (e.g., assume no damping when there is)?

In [ ]:
# === Exercise 1: Identify omega instead ===
# YOUR CODE HERE


In [ ]:
# === Exercise 2: Identify both parameters ===
# YOUR CODE HERE


In [ ]:
# === Exercise 3: Bootstrap uncertainty ===
# YOUR CODE HERE


In [ ]:
# === Exercise 4: Model mismatch ===
# YOUR CODE HERE


---
## Key Takeaways

### ✅ What We Learned

1. **PINNs excel at inverse problems**: Physics constraint regularizes parameter estimation
2. **Joint optimization works**: NN weights and physics parameters optimize together
3. **Robust to noise**: Physics provides prior knowledge that helps with noisy data
4. **Data efficiency**: Works with sparse observations if physics is correct

### ⚠️ Limitations

1. **Identifiability**: Some parameters may not be uniquely determinable
2. **Model mismatch**: Wrong physics → wrong parameters
3. **Multiple parameters**: Harder, may have multiple minima
4. **No uncertainty**: Basic PINN doesn't quantify confidence

### 💡 When to Use PINNs for Inverse Problems?

- **Parameter discovery** from experimental data
- **System identification** for dynamical systems
- **When classical methods need analytical derivatives** (PINN uses autodiff)
- **Multi-physics problems** where parameters appear in complex ways